# Módulo Automatización: Voz → Texto (Whisper)
**Objetivo:** Implementar un micro-módulo reutilizable que reciba un archivo de audio (wav/mp3), lo transcriba a texto usando Whisper y devuelva la transcripción via API.  
Arquitectura: **Hexagonal** (API / Application / Domain / Infrastructure).  
Esto facilita cambiar el motor (Whisper → Google Speech → Azure) sin tocar la lógica de la aplicación.


## Resumen rápido
- **Entrada:** archivo de audio (POST /automation/voice-to-text).
- **Salida:** JSON `{ "transcription": "texto..." }`.
- **Capas:**
  - `domain`: define el contrato (interface/protocol).
  - `infrastructure`: implementación concreta (Whisper).
  - `application`: lógica de la aplicación (usa la infraestructura).
  - `api`: rutas FastAPI que exponen la funcionalidad.
- **Requisitos:** Python 3.10+, pip, ffmpeg (para algunos formatos).


## Requisitos / Instalación
En el entorno virtual (recomendado):


In [ ]:
# Dependencias mínimas
pip install fastapi uvicorn "openai-whisper" torch python-multipart
# (Opcionales / para pruebas)
pip install requests gTTS pydub



**Notas**:
- `openai-whisper` usa PyTorch; en CPU funciona pero será más lento. En máquinas con GPU instala la versión de `torch` adecuada (consulta la página oficial de PyTorch para el comando `pip` correcto).
- Para procesar MP3/otros formatos `pydub` y `ffmpeg` pueden ser útiles. Instala `ffmpeg` en tu sistema si lo vas a usar.


## Estructura de archivos (dentro de `app/automation/`)
app/
 └── automation/
      ├── api/
      │    └── routes.py
      ├── application/
      │    └── automation_service.py
      ├── domain/
      │    └── automation_interface.py
      └── infrastructure/
           └── whisper_engine.py
Además deberías tener tu `main.py` donde incluyes el router (ya lo tienes).


### domain/automation_interface.py (código con comentario)

In [ ]:
"""
Esta clase define una INTERFAZ usando `Protocol`, que funciona como un contrato.
- `Protocol` en Python → permite declarar métodos que otras clases deben implementar,
  parecido a las interfaces en Java o C#.
- Aquí declaramos el método `transcribe(audio_path: str) -> str`, que recibe la ruta de un
  archivo de audio y devuelve el texto transcrito.

El `...` (ellipsis) significa:
- "Aquí no pongo implementación, solo marco que este método existe".
- Es equivalente (conceptualmente) a `pass`, pero se usa mucho en interfaces para dejar claro que
  este espacio NO se llena aquí, sino en la clase que implemente la interfaz.
- Sirve para que cualquier motor (Whisper, Google Speech, Azure, etc.) sepa qué función
  debe tener y qué debe devolver, sin importar cómo lo haga internamente.

Ventaja de esta separación:
- La lógica de negocio (nuestro sistema necesita convertir voz a texto) no depende
  de un motor específico.
- La implementación (cómo se logra esa conversión) puede cambiar libremente.
  Hoy usamos Whisper, mañana podríamos usar Google Speech, y no habría que modificar
  el resto del código, porque todos cumplen el mismo contrato (`transcribe`).
"""
from typing import Protocol

class VoiceToTextInterface(Protocol):
    def transcribe(self, audio_path: str) -> str:
        """Transcribe un archivo de audio y devuelve texto"""
        ...


## Errores comunes
- `ModuleNotFoundError: No module named 'whisper'` → revisa que `openai-whisper` esté instalado y Python esté usando el entorno correcto.
- Problemas de memoria al cargar modelos grandes → usa "base" o "tiny" o ejecuta en máquina con GPU.
- Audio no compatible / formato extraño → convierte a WAV (16k/44.1k) con `ffmpeg` o `pydub`.
- Si la transcripción tarda mucho → prueba con modelos más pequeños o haz pruebas con audios cortos.


## Consideraciones para producción / concurso
- **Modelo:** Para demo de concurso usa `base` (balance precisión/velocidad). Para mayor precisión en pruebas reales, `small` o `medium`.
- **Escalado:** Cargar un modelo por proceso es costoso; usar un servicio dedicado (microservicio) con colas si habrá muchas solicitudes simultáneas.
- **Persistencia:** Si deseas histórico, guarda transcripciones en una tabla `transcriptions` (id, original_filename, path_s3?, text, created_at, duration, confidence).
- **Almacenamiento:** Para archivos grandes o producción, no guardes en disco local; usa S3 / MinIO y procesa desde allí.
- **Seguridad:** Añadir autenticación/authorization (JWT) en las rutas.
- **Observabilidad:** Añadir logging, métricas (Prometheus), seguimiento de latencias.
- **Tolerancia a fallos:** manejo de timeouts, reintentos y circuit breakers si usas servicios externos.
- **Costos:** modelos grandes consumen RAM/CPU. Prueba trade-off.


## Demo sugerida para el concurso
1. Muestra Swagger: sube `test_audio.wav` y enseña la respuesta JSON.
2. Haz un demo en vivo: graba un clip corto (teléfono) y súbelo.
3. Explica la arquitectura hexagonal en una slide (domain/api/application/infrastructure).
4. Muestra cómo cambiar la implementación (breve slide: "si cambiamos Whisper por Google Speech, solo tocamos infraestructura").
5. Menciones técnicas: modelo usado, tiempo promedio de transcripción por audio de 10s (métricas).
6. Siguiente pasos (facturas + recordatorios): explica que la capa `application` es donde guardaremos y conectaremos a la BD.
